In [28]:
!pip install -r C:\Users\ezrag\OneDrive\Documents\GitHub\spotify-listening-data\requirements.txt

  Using cached certifi-2024.8.30-py3-none-any.whl.metadata (2.2 kB)
  Using cached idna-3.10-py3-none-any.whl.metadata (10 kB)
Using cached certifi-2024.8.30-py3-none-any.whl (167 kB)
Using cached idna-3.10-py3-none-any.whl (70 kB)


In [ ]:
import os
from dotenv import load_dotenv

# Load environment variables from .env file
load_dotenv()

SPOTIFY_CLIENT_ID = os.getenv('SPOTIFY_CLIENT_ID')
SPOTIFY_CLIENT_SECRET = os.getenv('SPOTIFY_CLIENT_SECRET')

In [ ]:
# Import necessary libraries
import pandas as pd
import json
import os
import random
from datetime import datetime, timedelta
from dotenv import load_dotenv
import threading
import queue
import requests
import time

In [31]:
# Function to get user ID from input
def get_user_id():
    user_id = input("Enter the user's ID: ").lower()
    return user_id


In [11]:
# Function to get the number of data chunks from input
def get_num_chunks():
    num_chunks = int(input("Enter the number of chunks: "))
    return num_chunks

In [12]:
# Function to read and process data from multiple JSON files
def read_and_process_data(user_id, num_chunks, base_path='wrapped_files/'):
    all_data = []
    
    for i in range(num_chunks):
        json_file = os.path.join(base_path, f'{user_id}_music_{i}.json')
        print(f"Checking for file: {json_file}")
        
        if not os.path.exists(json_file):
            print(f"File not found: {json_file}")
            continue
        
        print(f"Reading data from {json_file}")
        with open(json_file, 'r', encoding='utf-8') as file:
            data_list = json.load(file)
            all_data.extend(data_list)
    
    if not all_data:
        raise ValueError("No data files were found or all were empty.")
    
    df = pd.DataFrame(all_data)
    df['user_id'] = user_id
    df['endTime'] = pd.to_datetime(df['endTime'])
    
    print(f"Data read successfully for {len(df)} records.")
    return df


In [13]:
# Function to export data to a CSV file
def export_to_csv(df, user_id):
    csv_file = f'{user_id}_listening_data.csv'
    df.to_csv(csv_file, index=False)
    print(f"Data exported to {csv_file}")


In [14]:
# Function to track and save unique songs to a CSV file
def track_unique_songs(df, unique_songs_file):
    unique_songs = df[['trackName', 'artistName']].drop_duplicates()
    print(f"Tracking {len(unique_songs)} unique songs.")
    
    try:
        existing_songs = pd.read_csv(unique_songs_file)
        updated_songs = pd.concat([existing_songs, unique_songs]).drop_duplicates()
        print(f"Existing unique songs loaded, total unique songs now {len(updated_songs)}.")
    except FileNotFoundError:
        updated_songs = unique_songs
        print("Unique songs file not found. Creating a new one.")
    
    updated_songs.to_csv(unique_songs_file, index=False)
    print(f"Unique songs tracked and saved to {unique_songs_file}.")


In [ ]:
# Function to get Spotify access token using client credentials
def get_spotify_access_token(client_id, client_secret):
    auth_url = 'https://accounts.spotify.com/api/token'
    auth_response = requests.post(auth_url, {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
    })
    
    # Parse the authentication response and extract access token
    auth_response_data = auth_response.json()
    return auth_response_data['access_token']

In [ ]:
# Function to get song details from Spotify API using search query
def get_song_details(artist_name, track_name, access_token):
    search_url = 'https://api.spotify.com/v1/search'
    headers = {
        'Authorization': f'Bearer {access_token}'
    }
    params = {
        'q': f'artist:{artist_name} track:{track_name}',
        'type': 'track',
        'limit': 1
    }
    
    # Send request to Spotify API to search for the track
    response = requests.get(search_url, headers=headers, params=params)
    response_data = response.json()
    
    if response_data['tracks']['items']:
        track_info = response_data['tracks']['items'][0]
        song_details = {
            'spotify_id': track_info['id'],
            'album': track_info['album']['name'],
            'release_date': track_info['album']['release_date'],
            'popularity': track_info['popularity'],
            'duration_ms': track_info['duration_ms'],
            'track_number': track_info['track_number'],
            'album_artwork': track_info['album']['images'][0]['url'] if track_info['album']['images'] else None,
            'external_urls': track_info['external_urls']['spotify'],
            'artists_involved': ", ".join(artist['name'] for artist in track_info['artists'])
        }
        return song_details
    else:
        return None

In [47]:
# Worker function to process each song in the queue
def worker_thread(queue, unique_songs, access_token, export_interval, lock, start_time):
    while not queue.empty():
        index, row = queue.get()
        if pd.notna(row['spotify_id']):
            print(f"Skipping already updated song at index {index}.")
            queue.task_done()
            continue
        
        artist_name = row['artistName']
        track_name = row['trackName']
        song_details = get_song_details(artist_name, track_name, access_token)
        
        if song_details:
            with lock:
                unique_songs.at[index, 'spotify_id'] = song_details['spotify_id']
                unique_songs.at[index, 'album'] = song_details['album']
                unique_songs.at[index, 'release_date'] = song_details['release_date']
                unique_songs.at[index, 'popularity'] = song_details['popularity']
                unique_songs.at[index, 'duration_ms'] = song_details['duration_ms']
                unique_songs.at[index, 'track_number'] = song_details['track_number']
                unique_songs.at[index, 'album_artwork'] = song_details['album_artwork']
                unique_songs.at[index, 'external_urls'] = song_details['external_urls']
                unique_songs.at[index, 'artists_involved'] = song_details['artists_involved']
        
        if (index + 1) % export_interval == 0:
            with lock:
                print(f"Exporting data at index {index}. Elapsed time: {time.time() - start_time:.2f} seconds.")
                unique_songs.to_csv(unique_songs_file, index=False)
        
        queue.task_done()
        print(f"Processed index {index}")

In [48]:
# Main function to update unique songs table with Spotify info using threading
def update_unique_songs(unique_songs_file='unique_songs.csv', export_interval=50):
    # Load unique songs data from CSV file
    unique_songs = pd.read_csv(unique_songs_file)
    
    # Check if the columns already exist, if not, create them
    if 'spotify_id' not in unique_songs.columns:
        unique_songs['spotify_id'] = None
    if 'album' not in unique_songs.columns:
        unique_songs['album'] = None
    if 'release_date' not in unique_songs.columns:
        unique_songs['popularity'] = None
    if 'duration_ms' not in unique_songs.columns:
        unique_songs['duration_ms'] = None
    if 'track_number' not in unique_songs.columns:
        unique_songs['track_number'] = None
    if 'album_artwork' not in unique_songs.columns:
        unique_songs['album_artwork'] = None
    if 'external_urls' not in unique_songs.columns:
        unique_songs['external_urls'] = None
    if 'artists_involved' not in unique_songs.columns:
        unique_songs['artists_involved'] = None

    # Get Spotify access token
    access_token = get_spotify_access_token(SPOTIFY_CLIENT_ID, SPOTIFY_CLIENT_SECRET)
    
    # Create a queue and add songs to be processed
    q = queue.Queue()
    for index, row in unique_songs.iterrows():
        q.put((index, row))

    # Create a lock for thread-safe operations
    lock = threading.Lock()
    start_time = time.time()
    threads = []
    for _ in range(10):  # Adjust number of threads as needed
        thread = threading.Thread(target=worker_thread, args=(q, unique_songs, access_token, export_interval, lock, start_time))
        thread.start()
        threads.append(thread)
    
    # Wait for all threads to complete
    for thread in threads:
        thread.join()
    
    # Final export
    print(f"Final export. Total time taken: {time.time() - start_time:.2f} seconds.")
    unique_songs.to_csv(unique_songs_file, index=False)
    print(f"Unique songs table updated with Spotify info and saved to {unique_songs_file}.")


In [ ]:
# Function to fill in song info from unique songs database
def fill_song_info(listening_data, unique_songs):
    # Merge listening data with unique songs data on 'artistName' and 'trackName'
    filled_data = pd.merge(listening_data, unique_songs, on=['artistName', 'trackName'], how='left')
    return filled_data

In [53]:
# Function to read processed listening data
def read_processed_data(user_id):
    csv_file = f'{user_id}_listening_data.csv'  # Example file path, adjust as needed
    listening_data = pd.read_csv(csv_file)
    return listening_data

In [55]:
# Function to export filled listening data to a CSV file
def export_filled_data(filled_data, user_id):
    filled_csv_file = f'{user_id}_filled_listening_data.csv'
    filled_data.to_csv(filled_csv_file, index=False)
    print(f"Filled listening data exported to {filled_csv_file}")

In [18]:
# Execute the main steps to read data, export to CSV, and track unique songs
user_id = get_user_id()
num_chunks = get_num_chunks()
base_path = '../wrapped_files/'  # Adjusting the relative path based on the notebook location
unique_songs_file = 'unique_songs.csv'

try:
    df = read_and_process_data(user_id, num_chunks, base_path)
    export_to_csv(df, user_id)
    track_unique_songs(df, unique_songs_file)

    print("Data processing complete!")
except ValueError as e:
    print(e)

Checking for file: ../wrapped_files/samfa_music_0.json
Reading data from ../wrapped_files/samfa_music_0.json
Data read successfully for 6228 records.
Data exported to samfa_listening_data.csv
Tracking 3186 unique songs.
Existing unique songs loaded, total unique songs now 7485.
Unique songs tracked and saved to unique_songs.csv.
Data processing complete!


In [45]:
import requests
# Execute the function to update unique songs table with Spotify info
update_unique_songs('unique_songs.csv')

Skipping already updated song at index 0.
Skipping already updated song at index 1.
Skipping already updated song at index 2.
Skipping already updated song at index 3.
Skipping already updated song at index 4.
Skipping already updated song at index 5.
Skipping already updated song at index 6.
Skipping already updated song at index 7.
Skipping already updated song at index 8.
Skipping already updated song at index 9.
Skipping already updated song at index 10.
Skipping already updated song at index 11.
Skipping already updated song at index 12.
Skipping already updated song at index 13.
Skipping already updated song at index 14.
Skipping already updated song at index 15.
Skipping already updated song at index 16.
Skipping already updated song at index 17.
Skipping already updated song at index 18.
Skipping already updated song at index 19.
Skipping already updated song at index 20.
Skipping already updated song at index 21.
Skipping already updated song at index 22.
Skipping already upda

In [49]:
import pandas as pd

# Load the unique songs database
unique_songs_file = 'unique_songs.csv'
unique_songs = pd.read_csv(unique_songs_file)

# Sort the database by artistName
sorted_unique_songs = unique_songs.sort_values(by='artistName')

# Save the sorted database to a new CSV file
sorted_unique_songs_file = 'sorted_unique_songs.csv'
sorted_unique_songs.to_csv(sorted_unique_songs_file, index=False)

print(f"Sorted unique songs database saved to {sorted_unique_songs_file}.")

Sorted unique songs database saved to sorted_unique_songs.csv.


In [74]:
# Load unique songs data
unique_songs_file = 'unique_songs.csv'
unique_songs = pd.read_csv(unique_songs_file)

# Get user ID and read processed listening data
user_id = get_user_id()
try:
    listening_data = read_processed_data(user_id)
    
    # Fill in song info from unique songs database
    filled_listening_data = fill_song_info(listening_data, unique_songs)
    
    # Export the filled listening data to a new CSV file
    export_filled_data(filled_listening_data, user_id)

    print("Data processing complete!")
except FileNotFoundError:
    print(f"Processed data file not found for user ID: {user_id}")


Filled listening data exported to ezra_filled_listening_data.csv
Data processing complete!


# Analysis of Filled Listening Data

In [57]:
import pandas as pd

# Function to read filled listening data
def read_filled_listening_data(file_path):
    df = pd.read_csv(file_path)
    return df

In [58]:
# Function to print column names
def print_column_names(df):
    print("Column names in the filled listening data:")
    for column in df.columns:
        print(column)

In [63]:
# Function to calculate top listened-to artists by count
def top_artists_by_count(df, top_n=10):
    artist_count = df['artistName'].value_counts().head(top_n)
    return artist_count

In [68]:
# Function to calculate top listened-to artists by listening time
def top_artists_by_time(df, top_n=10):
    artist_time = df.groupby('artistName')['msPlayed'].sum().sort_values(ascending=False).head(top_n)
    # Convert milliseconds to seconds
    artist_time_seconds = artist_time / 1000
    return artist_time_seconds

In [70]:
# Function to calculate percentage listened for each track
def calculate_percentage_listened(df):
    df['percentage_listened'] = df['msPlayed'] / df['duration_ms']
    return df

In [71]:
# Function to calculate top listened-to artists by weighted listening time
def top_artists_by_weighted_time(df, top_n=10):
    artist_weighted_time = df.groupby('artistName')['percentage_listened'].sum().sort_values(ascending=False).head(top_n)
    return artist_weighted_time

In [75]:
# Get user name and construct the file path
user_name = get_user_id()
file_path = f'{user_name}_filled_listening_data.csv'

# Read the filled listening data
try:
    filled_listening_data = read_filled_listening_data(file_path)
    
    # Calculate percentage listened for each track
    filled_listening_data = calculate_percentage_listened(filled_listening_data)
    
    # Calculate top listened-to artists by count
    top_artists_count = top_artists_by_count(filled_listening_data)
    print("Top listened-to artists by count:")
    for artist, count in top_artists_count.items():
        print(f"{artist}: {count} plays")
    
    # Calculate top listened-to artists by listening time
    top_artists_time = top_artists_by_time(filled_listening_data)
    print("\nTop listened-to artists by listening time:")
    for artist, time_sec in top_artists_time.items():
        time_min = time_sec / 60
        print(f"{artist}: {time_sec:.2f} seconds ({time_min:.2f} minutes)")
    
    # Calculate top listened-to artists by weighted listening time
    top_artists_weighted_time = top_artists_by_weighted_time(filled_listening_data)
    print("\nTop listened-to artists by weighted listening time:")
    for artist, weighted_time in top_artists_weighted_time.items():
        print(f"{artist}: {weighted_time:.2f} total weighted listens")

    print("Data analysis complete!")
except FileNotFoundError:
    print(f"File not found: {file_path}")


Top listened-to artists by count:
Unknown Artist: 612 plays
Crazy Ex-Girlfriend Cast: 451 plays
Lady Gaga: 194 plays
Bob's Burgers: 193 plays
RuPaul: 175 plays
Dai: 167 plays
Nicki Minaj: 152 plays
Britney Spears: 152 plays
Katy Perry: 145 plays
Chappell Roan: 139 plays

Top listened-to artists by listening time:
Crazy Ex-Girlfriend Cast: 39934.49 seconds (665.57 minutes)
Lady Gaga: 26978.19 seconds (449.64 minutes)
RuPaul: 24198.67 seconds (403.31 minutes)
Britney Spears: 20412.78 seconds (340.21 minutes)
Nicki Minaj: 19403.27 seconds (323.39 minutes)
Katy Perry: 18128.45 seconds (302.14 minutes)
Taylor Swift: 17881.06 seconds (298.02 minutes)
Dai: 17375.54 seconds (289.59 minutes)
Glee Cast: 16480.49 seconds (274.67 minutes)
Chappell Roan: 16345.20 seconds (272.42 minutes)

Top listened-to artists by weighted listening time:
Crazy Ex-Girlfriend Cast: 307.17 total weighted listens
RuPaul: 116.76 total weighted listens
Lady Gaga: 116.38 total weighted listens
Bob's Burgers: 110.04 tota

In [103]:
import pandas as pd

# Function to find the most common album art for each artist
def get_most_common_album_art(df, top_artists):
    album_art = {}
    for artist in top_artists:
        artist_data = df[df['artistName'] == artist]
        most_common_art = artist_data['album_artwork'].mode().iloc[0]
        album_art[artist] = most_common_art
    return album_art


In [104]:
from PIL import Image, ImageDraw, ImageFont

# Function to create and save layout images
def create_layout_image(title, top_artists, album_art, file_path):
    width, height = 1080, 1920  # Size of an Instagram story
    image = Image.new('RGB', (width, height), color='black')
    draw = ImageDraw.Draw(image)

    # Load a font
    font = ImageFont.truetype("arial.ttf", 40)
    title_font = ImageFont.truetype("arial.ttf", 60)
    
    # Title
    draw.text((width / 2, 50), title, font=title_font, fill="white", anchor="mm")

    # Starting positions
    y_offset = 150
    x_offset = 50

    # Draw the section
    for rank, (artist, value) in enumerate(top_artists.items(), start=1):
        if artist not in album_art:
            continue
        # Draw album art
        art = Image.open(album_art[artist]).resize((100, 100))
        image.paste(art, (x_offset, y_offset))
        draw.text((x_offset + 120, y_offset), f"{rank}. {artist}: {value}", font=font, fill="white")
        y_offset += 120

    # Save Image
    image.save(file_path)
    print(f"Layout image saved to {file_path}")


In [105]:
# Function to fetch and download all popular album art for each artist
def download_all_album_art(df, top_artists):
    album_art = {}
    for artist in top_artists:
        artist_data = df[df['artistName'] == artist]
        if artist_data.empty:
            continue
        # Get the most frequent album art URLs
        art_urls = artist_data['album_artwork'].value_counts().index.tolist()
        downloaded = False
        for art_url in art_urls:
            try:
                # Download the image
                response = requests.get(art_url)
                img = Image.open(BytesIO(response.content))
                
                # Save the image locally in the albums folder
                img_path = os.path.join(albums_folder, f'{artist}_album_art.jpg')
                img.save(img_path)
                
                # Update dictionary with local path
                album_art[artist] = img_path
                downloaded = True
                break  # Stop once an image is successfully downloaded
            except Exception as e:
                print(f"Error downloading {art_url} for {artist}: {e}")
                continue
        if not downloaded:
            print(f"Could not download album art for {artist}")
    return album_art


In [ ]:
# Generate abstract background with dynamic colors and Perlin noise
background = generate_abstract_background_with_noise(1080, 1920)

# Load unique songs data and filled listening data
unique_songs_file = 'unique_songs.csv'
unique_songs = pd.read_csv(unique_songs_file)

user_name = get_user_id()
file_path = f'wrapped_project/wrapped_notebooks/{user_name}_filled_listening_data.csv'

try:
    filled_listening_data = read_filled_listening_data(file_path)
    
    # Calculate percentage listened for each track
    filled_listening_data = calculate_percentage_listened(filled_listening_data)
    
    # Calculate top listened-to artists by count
    top_artists_count = top_artists_by_count(filled_listening_data).head(5)
    
    # Calculate top listened-to artists by listening time
    top_artists_time = top_artists_by_time(filled_listening_data).head(5)
    
    # Calculate top listened-to artists by weighted listening time
    top_artists_weighted_time = top_artists_by_weighted_time(filled_listening_data).head(5)
    
    # Combine all top artists to ensure all album art is downloaded
    all_top_artists = top_artists_count.index.union(top_artists_time.index).union(top_artists_weighted_time.index)
    
    # Download the most common album art for each artist
    album_art = download_all_album_art(filled_listening_data, all_top_artists)
    
    # Create layout images with user ID in file name and abstract background
    create_layout_image("Top Artists by Count", top_artists_count, album_art, f"{user_name}_spotify_wrapped_top_artists_count.png", user_name, background)
    create_layout_image("Top Artists by Listening Time (minutes)", {k: v / 60 for k, v in top_artists_time.items()}, album_art, f"{user_name}_spotify_wrapped_top_artists_time.png", user_name, background)
    create_layout_image("Top Artists by Weighted Listening Time", top_artists_weighted_time, album_art, f"{user_name}_spotify_wrapped_top_artists_weighted_time.png", user_name, background)
    
    print("Data processing and layout creation complete!")
except FileNotFoundError:
    print(f"File not found: {file_path}")


Layout image saved to ezra_spotify_wrapped_top_artists_count.png
Layout image saved to ezra_spotify_wrapped_top_artists_time.png
Layout image saved to ezra_spotify_wrapped_top_artists_weighted_time.png
Data processing and layout creation complete!


In [122]:
!pip install noise

  Using cached noise-1.2.2.zip (132 kB)
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Running setup.py clean for noise
Failed to build noise


  error: subprocess-exited-with-error
  
  × python setup.py bdist_wheel did not run successfully.
  │ exit code: 1
  ╰─> [12 lines of output]
      running bdist_wheel
      running build
      running build_py
      creating build\lib.win-amd64-cpython-313\noise
      copying .\perlin.py -> build\lib.win-amd64-cpython-313\noise
      copying .\shader.py -> build\lib.win-amd64-cpython-313\noise
      copying .\shader_noise.py -> build\lib.win-amd64-cpython-313\noise
      copying .\test.py -> build\lib.win-amd64-cpython-313\noise
      copying .\__init__.py -> build\lib.win-amd64-cpython-313\noise
      running build_ext
      building 'noise._simplex' extension
      error: Microsoft Visual C++ 14.0 or greater is required. Get it with "Microsoft C++ Build Tools": https://visualstudio.microsoft.com/visual-cpp-build-tools/
      [end of output]
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  ERROR: Failed building wheel for noise
ERROR: ERROR

In [120]:
import random
import numpy as np
import matplotlib.pyplot as plt
from PIL import Image

# Function to generate random colors within a harmonious range
def generate_random_color():
    return (random.randint(100, 255), random.randint(100, 255), random.randint(100, 255))

# Function to generate an abstract background with dynamic colors
def generate_abstract_background(width, height):
    # Generate random start and end colors
    start_color = generate_random_color()
    end_color = generate_random_color()
    
    # Create a meshgrid
    x = np.linspace(-5, 5, width)
    y = np.linspace(-5, 5, height)
    X, Y = np.meshgrid(x, y)
    
    # Generate abstract pattern using sine and cosine functions
    Z = np.sin(X**2 + Y**2) * np.cos(Y**2 - X**2)
    
    # Plot and save the background with dynamic colors
    plt.figure(figsize=(width/100, height/100), dpi=100)
    plt.imshow(Z, cmap='coolwarm', interpolation='bilinear')
    plt.axis('off')
    plt.savefig('abstract_background.png', bbox_inches='tight', pad_inches=0)
    plt.close()

    # Open and return the background image
    background = Image.open('abstract_background.png')
    return background

import noise

# Function to generate Perlin noise
def generate_perlin_noise(width, height, scale=100):
    shape = (width, height)
    world = np.zeros(shape)
    for i in range(shape[0]):
        for j in range(shape[1]):
            world[i][j] = noise.pnoise2(i / scale, j / scale, octaves=6, persistence=0.5, lacunarity=2.0, repeatx=1024, repeaty=1024, base=42)
    
    max_val = np.max(world)
    min_val = np.min(world)
    norm_world = (world - min_val) / (max_val - min_val)
    return norm_world

# Function to generate an abstract background with Perlin noise
def generate_abstract_background_with_noise(width, height):
    noise_pattern = generate_perlin_noise(width, height)
    
    # Plot and save the background with Perlin noise
    plt.figure(figsize=(width/100, height/100), dpi=100)
    plt.imshow(noise_pattern, cmap='plasma', interpolation='bilinear')
    plt.axis('off')
    plt.savefig('abstract_background.png', bbox_inches='tight', pad_inches=0)
    plt.close()

    # Open and return the background image
    background = Image.open('abstract_background.png')
    return background


# Function to draw wrapped text
def draw_wrapped_text(draw, text, position, font, max_width, fill):
    lines = []
    words = text.split()
    while words:
        line = ''
        while words and font.getbbox(line + words[0])[2] <= max_width:
            line += (words.pop(0) + ' ')
        lines.append(line)
    y_offset = position[1]
    for line in lines:
        draw.text((position[0], y_offset), line, font=font, fill=fill)
        y_offset += font.getbbox(line)[3]  # Use getbbox for line height
    return y_offset


# Function to create and save layout images
def create_layout_image(title, top_artists, album_art, file_path, user_id, background):
    width, height = background.size
    image = background.copy()
    draw = ImageDraw.Draw(image)

    # Load a font
    font = ImageFont.truetype("arial.ttf", 40)
    title_font = ImageFont.truetype("arial.ttf", 60)
    user_id_font = ImageFont.truetype("arial.ttf", 30)
    
    # Title
    draw.text((width / 2, 50), title, font=title_font, fill="white", anchor="mm")
    draw.text((width / 2, 150), f"User: {user_id}", font=user_id_font, fill="white", anchor="mm")

    # Starting positions
    y_offset = 250
    x_offset = 50

    # Draw the section
    for rank, (artist, value) in enumerate(top_artists.items(), start=1):
        if artist not in album_art:
            continue
        # Draw album art
        art = Image.open(album_art[artist]).resize((100, 100))
        image.paste(art, (x_offset, y_offset))
        
        text = f"{rank}. {artist}: {value}"
        draw_wrapped_text(draw, text, (x_offset + 120, y_offset), font, max_width=width - x_offset - 120, fill="white")
        y_offset += 120

    # Save Image
    image.save(file_path)
    print(f"Layout image saved to {file_path}")



ModuleNotFoundError: No module named 'noise'